In [1]:
import os
import pandas as pd
import re
import string
from tqdm import tqdm
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
import logging
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import random
from collections import defaultdict
from multiprocessing import Pool
import functools
from scipy import spatial
import ast


import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')

pd.set_option('display.max_columns', None)

C:\Users\Karl Zhang\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Karl Zhang\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Karl Zhang\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to C:\Users\Karl
[nltk_data]     Zhang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Karl
[nltk_data]     Zhang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Feature extraction for food
This is the method from Roald.

In [2]:
stop_words = set(stopwords.words('english')) 

punctuation_table = str.maketrans({key: None for key in string.punctuation})
sno = SnowballStemmer('english')

def normalize_text(raw_text):
    try:
        word_list = word_tokenize(raw_text)
        normalized_sentence = []
        for w in word_list:
            try:
                w = str(w)
                lower_case_word = str.lower(w)
                stemmed_word = sno.stem(lower_case_word)
                no_punctuation = stemmed_word.translate(punctuation_table)
                if len(no_punctuation) > 1 and no_punctuation not in stop_words:
                    normalized_sentence.append(no_punctuation)
            except:
                continue
        return normalized_sentence
    except:
        return ''

In [3]:
food_trigram_model = Phraser.load('food_trigram_model.pkl')

In [4]:
wine_word2vec_model = Word2Vec.load("word2vec.model")

In [5]:
foods = pd.read_csv('list_of_foods.csv')
foods_list = list(foods['Food'])
foods_list_normalized = [normalize_text(f) for f in foods_list]
foods_list_preprocessed = [food_trigram_model[f][0] for f in foods_list_normalized]
foods_list_preprocessed = list(set(foods_list_preprocessed))

In [6]:
foods_vecs = dict()

word_vectors = wine_word2vec_model.wv
for f in foods_list_preprocessed:
    try:
        food_vec = word_vectors[f]
        foods_vecs[f] = food_vec
    except:
        continue

In [7]:
core_tastes_revised = {'weight': ['heavy', 'cassoulet', 'cassoulet', 'full_bodied', 'thick', 'milk', 'fat', 'mincemeat', 'steak', 'bold', 'pizza', 'pasta', 'creamy', 'bread'],
                       'sweet': ['sweet', 'sugar', 'cake', 'mango', 'stevia'], 
                       'acid': ['acid', 'sour', 'vinegar', 'yoghurt', 'cevich', 'cevich'],
                       'salt': ['salty', 'salty', 'parmesan', 'oyster', 'pizza', 'bacon', 'cured_meat', 'potato_chip'], 
                       'piquant': ['spicy'], 
                       'fat': ['fat', 'creamy', 'cassoulet', 'foie_gras', 'cake', 'foie_gras', 'brie', 'carbonara'], 
                       'bitter': ['bitter', 'kale']
                      }

average_taste_vecs = dict()
core_tastes_distances = dict()
for taste, keywords in core_tastes_revised.items():
    
    all_keyword_vecs = []
    for keyword in keywords:
        c_vec = word_vectors[keyword]
        all_keyword_vecs.append(c_vec)
    
    avg_taste_vec = np.average(all_keyword_vecs, axis=0)
    average_taste_vecs[taste] = avg_taste_vec
        
    taste_distances = dict()
    for k, v in foods_vecs.items():
        similarity = 1- spatial.distance.cosine(avg_taste_vec, v)
        taste_distances[k] = similarity
        
    core_tastes_distances[taste] = taste_distances      

In [8]:
food_nonaroma_infos = dict()
# for each core taste, identify the food item that is farthest and closest. We will need this to create a normalized scale between 0 and 1
for key, value in core_tastes_revised.items():
    dict_taste = dict()
    farthest = min(core_tastes_distances[key], key=core_tastes_distances[key].get)
    farthest_distance = core_tastes_distances[key][farthest]
    closest = max(core_tastes_distances[key], key=core_tastes_distances[key].get)
    closest_distance = core_tastes_distances[key][closest]
    print(key, farthest, closest)
    dict_taste['farthest'] = farthest_distance
    dict_taste['closest'] = closest_distance
    dict_taste['average_vec'] = average_taste_vecs[key]
    food_nonaroma_infos[key] = dict_taste

weight dragonfruit pizza
sweet venison honey
acid hungri tart
salt ice bacon
piquant gelatin teriyaki
fat pain cake
bitter biscuit kale


In [9]:
food_nonaroma_infos_df = pd.DataFrame(food_nonaroma_infos).T

In [10]:
food_nonaroma_infos_df.to_csv('average_nonaroma_vectors.csv')

In [11]:
food_nonaroma_infos = pd.read_csv('average_nonaroma_vectors.csv', index_col='Unnamed: 0')

In [12]:
food_nonaroma_infos

,farthest,closest,average_vec
weight,-0.110404,0.543864,[ 0.24076359 0.43425325 -0.04434961 0.090069...
sweet,-0.184637,0.560057,[-6.23854458e-01 1.26031005e+00 3.08796410e-...
acid,-0.122985,0.598332,[ 0.8224229 0.5588506 0.44359994 -0.222883...
salt,-0.135441,0.613997,[ 1.04816127e+00 6.13947153e-01 -3.70970070e-...
piquant,-0.153211,0.275217,[ 0.82449615 0.17294654 0.5056956 0.997986...
fat,-0.085688,0.578953,[ 6.1073273e-02 4.0769193e-01 -8.8821433e-02 ...
bitter,-0.166239,0.532240,[ 6.24758244e-01 1.39185119e+00 2.27915198e-...


In [13]:
# this function scales each nonaroma between 0 and 1
def minmax_scaler(val, minval, maxval):
    val = max(min(val, maxval), minval)
    normalized_val = (val - minval)/(maxval - minval)
    return normalized_val

# this function makes sure that a scaled value (between 0 and 1) is returned for a food nonaroma
def check_in_range(label_range_dict, value):
    for label, value_range_tuple in label_range_dict.items():
        lower_end = value_range_tuple[0]
        upper_end = value_range_tuple[1]
        if value >= lower_end and value <= upper_end:
            return label
        else:
            continue

# this function calculates the average word embedding of all foods supplied as input
def calculate_avg_food_vec(sample_foods):
    sample_food_vecs = []
    for s in sample_foods:
        sample_food_vec = word_vectors[s]
        sample_food_vecs.append(sample_food_vec)
    sample_food_vecs_avg = np.average(sample_food_vecs, axis=0)
    return sample_food_vecs_avg

# this function returns two things: a score (between 0 and 1) and a normalized value (integer between 1 and 4) for a given nonaroma
def nonaroma_values(nonaroma, average_food_embedding):
    average_taste_vec = food_nonaroma_infos.at[nonaroma, 'average_vec']
    average_taste_vec = re.sub('\s+', ',', average_taste_vec)
    average_taste_vec = average_taste_vec.replace('[,', '[')
    average_taste_vec  = np.array(ast.literal_eval(average_taste_vec))

    similarity = 1 - spatial.distance.cosine(average_taste_vec, average_food_embedding)
    # scale the similarity using our minmax scaler
    scaled_similarity = minmax_scaler(similarity, food_nonaroma_infos.at[nonaroma, 'farthest'], 
                                      food_nonaroma_infos.at[nonaroma, 'closest'])
    standardized_similarity = check_in_range(food_weights[nonaroma], scaled_similarity)
    similarity_and_scalar = (scaled_similarity, standardized_similarity)
    return similarity_and_scalar

# this function loops through the various nonaromas, returning the nonaroma scores & normalized values, the body/weight of the food and the average food embedding 
def return_all_food_values(sample_foods):
    food_nonaromas = dict()
    average_food_embedding = calculate_avg_food_vec(sample_foods)
    for nonaroma in ['sweet', 'acid', 'salt', 'piquant', 'fat', 'bitter']:
        food_nonaromas[nonaroma] = nonaroma_values(nonaroma, average_food_embedding)
    food_weight = nonaroma_values('weight', average_food_embedding)
    return food_nonaromas, food_weight, average_food_embedding

In [14]:
# this is the mapping of food similarities to the normalized ordinal integer values between 1 and 4
food_weights = {
    'weight': {1: (0, 0.3), 2: (0.3, 0.5), 3: (0.5, 0.7), 4: (0.7, 1)},
    'sweet': {1: (0, 0.45), 2: (0.45, 0.6), 3: (0.6, 0.8), 4: (0.8, 1)},
    'acid': {1: (0, 0.4), 2: (0.4, 0.55), 3: (0.55, 0.7), 4: (0.7, 1)},
    'salt': {1: (0, 0.3), 2: (0.3, 0.55), 3: (0.55, 0.8), 4: (0.8, 1)},
    'piquant': {1: (0, 0.4), 2: (0.4, 0.6), 3: (0.6, 0.8), 4: (0.8, 1)},
    'fat': {1: (0, 0.4), 2: (0.4, 0.5), 3: (0.5, 0.6), 4: (0.6, 1)},
    'bitter': {1: (0, 0.3), 2: (0.3, 0.5), 3: (0.5, 0.65), 4: (0.65, 1)}
}

In [18]:
pair_df = pd.read_csv('pairing_food_append_wine_taste.csv')
pair_df = pair_df.drop(columns=['Unnamed: 0'])
pair_df.head(3)

,Pairing Food,Grape,Popularity,Critics' Score,Avg. Price kr / 750ml,Wine,Region,Country,name_pair,sweetness,acidity,piquancy,fattiness,bitterness,Alcohol,Price_Review,Vintage,name_wine_review
0,beef and venison,Bordeaux Blend Red,1st,96 / 100,"7,926",Chateau Mouton Rothschild,Pauillac,France,Bordeaux Blend Red Chateau Mouton Rothschild P...,-27.316847,-0.000972,-5.65303,-4.383264,-10.966054,14%,$675,2016.0,Bordeaux-style Red Blend ChÃ¢teau Mouton Roths...
1,beef and venison,Bordeaux Blend Red,3rd,96 / 100,"10,564",Chateau Lafite Rothschild,Pauillac,France,Bordeaux Blend Red Chateau Lafite Rothschild P...,-0.000151,-0.000972,-5.65303,-4.383264,-10.966054,NaN,$869,2016.0,Bordeaux-style Red Blend ChÃ¢teau Lafite Roths...
2,beef and venison,Merlot,4th,96 / 100,"47,971",Petrus,Pomerol,France,Merlot Petrus Pomerol France,-0.000151,-3.860945,-5.65303,-4.383264,27.357079,14%,$65,2014.0,Merlot ChÃ¢teau Fayat Pomerol France


In [19]:
pair_df['Pairing Food'].unique()

array(['beef and venison', 'blue cheeses', 'brie and camembert',
       'cakes and cream', 'cheddar and gruyere', 'chicken and turkey',
       'chilis and hot spicy foods', 'chocolate and caramel',
       'duck, goose and game birds', 'fruit-based desserts',
       "goats' cheese and feta", 'lamb', 'manchego and parmesan',
       'meaty and oily fish', 'mushrooms', 'pork, ham and cold meats',
       'root vegetables and squashes', 'salads and green vegetables',
       'shellfish, crab and lobster', 'tomato-based dishes', 'white fish'],
      dtype=object)

In [29]:
food_beef = ['beef']
food_nonaromas_beef_and_vension, food_weight, aroma_embedding = return_all_food_values(food_beef)
food_nonaromas_beef_and_vension

{'sweet': (0.14189681132049217, 1),
 'acid': (0.3181446620030892, 1),
 'salt': (0.6552644575786216, 3),
 'piquant': (0.4730964736715084, 2),
 'fat': (0.42086276068404743, 2),
 'bitter': (0.2282057906088553, 1)}

In [33]:
food_blue_cheese = ['blue', 'cheese']
food_nonaromas_blue_cheese, food_weight, aroma_embedding = return_all_food_values(food_blue_cheese)
food_nonaromas_blue_cheese

{'sweet': (0.3561509274468511, 1),
 'acid': (0.48954826863947964, 2),
 'salt': (0.794095945943736, 3),
 'piquant': (0.5442537591518815, 2),
 'fat': (0.6118928098591325, 4),
 'bitter': (0.34129564233626464, 2)}

In [87]:
food_brie = ['brie', 'camembert']
food_nonaromas_brie, food_weight, aroma_embedding = return_all_food_values(food_brie)
food_nonaromas_brie

{'sweet': (0.44752871316299375, 1),
 'acid': (0.4732374658476264, 2),
 'salt': (0.8269178097409339, 4),
 'piquant': (0.5804622838269446, 2),
 'fat': (0.874639255187634, 4),
 'bitter': (0.44008251165727613, 2)}

In [88]:
food_cake = ['cake', 'cream']
food_nonaromas_cake, food_weight, aroma_embedding = return_all_food_values(food_cake)
food_nonaromas_cake

{'sweet': (1.0, 4),
 'acid': (0.5398360444227153, 2),
 'salt': (0.36940689218868045, 2),
 'piquant': (0.4043840806390277, 2),
 'fat': (0.9831584853419457, 4),
 'bitter': (0.5535153083946551, 3)}

In [89]:
food_cheddar = ['cheddar']
food_nonaromas_cheddar, food_weight, aroma_embedding = return_all_food_values(food_cheddar)
food_nonaromas_cheddar

{'sweet': (0.33049645627921886, 1),
 'acid': (0.46982335962171695, 2),
 'salt': (0.8239100300075216, 4),
 'piquant': (0.5459273538390678, 2),
 'fat': (0.37280288881269075, 1),
 'bitter': (0.39679119061213425, 2)}

In [90]:
food_chicken = ['chicken', 'turkey']
food_nonaromas_chicken, food_weight, aroma_embedding = return_all_food_values(food_chicken)
food_nonaromas_chicken

{'sweet': (0.10357514003649179, 1),
 'acid': (0.30835336383947537, 1),
 'salt': (0.6163068269126257, 3),
 'piquant': (0.4584774697842983, 2),
 'fat': (0.4604437268746159, 2),
 'bitter': (0.12284443616566292, 1)}

In [91]:
food_spicy = ['spicy']
food_nonaromas_spicy, food_weight, aroma_embedding = return_all_food_values(food_spicy)
food_nonaromas_spicy

{'sweet': (0.3696166277970507, 1),
 'acid': (0.5984811130561692, 3),
 'salt': (0.5137539752638228, 2),
 'piquant': (1.0, 4),
 'fat': (0.27171653135120505, 1),
 'bitter': (0.44757731955647717, 2)}

In [92]:
food_chocolate = ['chocolate', 'caramel']
food_nonaromas_chocolate, food_weight, aroma_embedding = return_all_food_values(food_chocolate)
food_nonaromas_chocolate

{'sweet': (0.9619885370890124, 4),
 'acid': (0.5346639758154086, 2),
 'salt': (0.34472727025370425, 2),
 'piquant': (0.5446808439097817, 2),
 'fat': (0.6016893690396977, 4),
 'bitter': (0.6046423661708037, 3)}

In [93]:
food_duck = ['duck']
food_nonaromas_duck, food_weight, aroma_embedding = return_all_food_values(food_duck)
food_nonaromas_duck

{'sweet': (0.006519498643254949, 1),
 'acid': (0.14808009760578617, 1),
 'salt': (0.3354980267619829, 2),
 'piquant': (0.3341350524962817, 1),
 'fat': (0.26348944259850626, 1),
 'bitter': (0.0617347617266899, 1)}

In [94]:
food_fruit = ['fruit', 'dessert']
food_nonaromas_fruit, food_weight, aroma_embedding = return_all_food_values(food_fruit)
food_nonaromas_fruit

{'sweet': (0.978387497024766, 4),
 'acid': (0.6293638409611867, 3),
 'salt': (0.42815593983853517, 2),
 'piquant': (0.42915533347168816, 2),
 'fat': (0.7217314266133997, 4),
 'bitter': (0.5908920905401812, 3)}

In [95]:
food_feta = ['goat', 'cheese', 'feta']
food_nonaromas_feta, food_weight, aroma_embedding = return_all_food_values(food_feta)
food_nonaromas_feta

{'sweet': (0.2761369535840878, 1),
 'acid': (0.5709529035105834, 3),
 'salt': (1.0, 4),
 'piquant': (0.677527011966388, 3),
 'fat': (0.7577317609516822, 4),
 'bitter': (0.4364221261900417, 2)}

In [101]:
food_lamb = ['lamb']
food_nonaromas_lamb, food_weight, aroma_embedding = return_all_food_values(food_lamb)
food_nonaromas_lamb

{'sweet': (0.14689989733093714, 1),
 'acid': (0.2997187967763121, 1),
 'salt': (0.43185307605279594, 2),
 'piquant': (0.38606852823728444, 1),
 'fat': (0.2929943318067957, 1),
 'bitter': (0.19335981965436308, 1)}

In [102]:
food_manchego = ['manchego', 'parmesan']
food_nonaromas_manchego, food_weight, aroma_embedding = return_all_food_values(food_manchego)
food_nonaromas_manchego

{'sweet': (0.3540694814984046, 1),
 'acid': (0.45162284884951104, 2),
 'salt': (1.0, 4),
 'piquant': (0.7049649581010795, 3),
 'fat': (0.6297846027058643, 4),
 'bitter': (0.47389320995147527, 2)}

In [103]:
food_meaty_fish = ['meaty', 'oily', 'fish']
food_nonaromas_meaty_fish, food_weight, aroma_embedding = return_all_food_values(food_meaty_fish)
food_nonaromas_meaty_fish

{'sweet': (0.3202083886685246, 1),
 'acid': (0.6615979737723163, 3),
 'salt': (0.7957911707831249, 3),
 'piquant': (0.7640370104140831, 3),
 'fat': (0.5989390316912432, 3),
 'bitter': (0.5537217723742949, 3)}

In [104]:
food_mushroom = ['mushroom']
food_nonaromas_mushroom, food_weight, aroma_embedding = return_all_food_values(food_mushroom)
food_nonaromas_mushroom

{'sweet': (0.361848989146613, 1),
 'acid': (0.479173818922394, 2),
 'salt': (0.7496693811845111, 3),
 'piquant': (0.5399000867685624, 2),
 'fat': (0.43088640507474885, 2),
 'bitter': (0.5209780065296126, 3)}

In [105]:
food_pork = ['pork', 'ham', 'cold', 'meat']
food_nonaromas_pork, food_weight, aroma_embedding = return_all_food_values(food_pork)
food_nonaromas_pork

{'sweet': (0.31027021038634073, 1),
 'acid': (0.5597675761205688, 3),
 'salt': (0.8700898812079234, 4),
 'piquant': (0.6295898342339498, 3),
 'fat': (0.6636865853814002, 4),
 'bitter': (0.3264533391751719, 2)}

In [106]:
food_root = ['root', 'squash']
food_nonaromas_root, food_weight, aroma_embedding = return_all_food_values(food_root)
food_nonaromas_root

{'sweet': (0.42822638614486414, 1),
 'acid': (0.3959429097602086, 1),
 'salt': (0.28749200998007457, 1),
 'piquant': (0.3203732204110389, 1),
 'fat': (0.25390341769213076, 1),
 'bitter': (0.5044450157468893, 3)}

In [107]:
food_salad = ['salad', 'green']
food_nonaromas_salad, food_weight, aroma_embedding = return_all_food_values(food_salad)
food_nonaromas_salad

{'sweet': (0.47989491174210985, 2),
 'acid': (0.7278642649875963, 4),
 'salt': (0.597343756936542, 3),
 'piquant': (0.6360630099264419, 3),
 'fat': (0.40935283955266005, 2),
 'bitter': (0.7332720299365881, 4)}

In [108]:
food_shellfish = ['shellfish', 'crab', 'lobster']
food_nonaromas_shellfish, food_weight, aroma_embedding = return_all_food_values(food_shellfish)
food_nonaromas_shellfish

{'sweet': (0.22437525283996917, 1),
 'acid': (0.4497198446719201, 2),
 'salt': (0.8525323433010747, 4),
 'piquant': (0.5834674660754088, 2),
 'fat': (0.7013883705494809, 4),
 'bitter': (0.19277164838102537, 1)}

In [109]:
food_tomato = ['tomato']
food_nonaromas_tomato, food_weight, aroma_embedding = return_all_food_values(food_tomato)
food_nonaromas_tomato

{'sweet': (0.4765833638726764, 2),
 'acid': (0.7596654492321429, 4),
 'salt': (0.7186933433053033, 3),
 'piquant': (0.6381221551947095, 3),
 'fat': (0.383053442232683, 1),
 'bitter': (0.6509030771098047, 4)}

In [110]:
food_white_fish = ['white', 'fish']
food_nonaromas_white_fish, food_weight, aroma_embedding = return_all_food_values(food_white_fish)
food_nonaromas_white_fish

{'sweet': (0.39061669372189767, 1),
 'acid': (0.5242231768059176, 2),
 'salt': (0.5495168175988254, 2),
 'piquant': (0.372132901898822, 1),
 'fat': (0.5151510563577111, 3),
 'bitter': (0.3689318223691101, 2)}

In [112]:
sweetness_list_food = []
acidity_list_food = []
salinity_list_food = []
piquancy_list_food = []
fattiness_list_food = []
bitterness_list_food = []

for index, row in tqdm(pair_df.iterrows()):
    food_name = row['Pairing Food']
    
    if food_name == 'beef and venison':
        sweetness_list_food.append(food_nonaromas_beef_and_vension['sweet'][0])
        acidity_list_food.append(food_nonaromas_beef_and_vension['acid'][0])
        salinity_list_food.append(food_nonaromas_beef_and_vension['salt'][0])
        piquancy_list_food.append(food_nonaromas_beef_and_vension['piquant'][0])
        fattiness_list_food.append(food_nonaromas_beef_and_vension['fat'][0])
        bitterness_list_food.append(food_nonaromas_beef_and_vension['bitter'][0])
    if food_name == 'blue cheeses':
        sweetness_list_food.append(food_nonaromas_blue_cheese['sweet'][0])
        acidity_list_food.append(food_nonaromas_blue_cheese['acid'][0])
        salinity_list_food.append(food_nonaromas_blue_cheese['salt'][0])
        piquancy_list_food.append(food_nonaromas_blue_cheese['piquant'][0])
        fattiness_list_food.append(food_nonaromas_blue_cheese['fat'][0])
        bitterness_list_food.append(food_nonaromas_blue_cheese['bitter'][0])
    if food_name == 'brie and camembert':
        sweetness_list_food.append(food_nonaromas_brie['sweet'][0])
        acidity_list_food.append(food_nonaromas_brie['acid'][0])
        salinity_list_food.append(food_nonaromas_brie['salt'][0])
        piquancy_list_food.append(food_nonaromas_brie['piquant'][0])
        fattiness_list_food.append(food_nonaromas_brie['fat'][0])
        bitterness_list_food.append(food_nonaromas_brie['bitter'][0])
    if food_name == 'cakes and cream':
        sweetness_list_food.append(food_nonaromas_cake['sweet'][0])
        acidity_list_food.append(food_nonaromas_cake['acid'][0])
        salinity_list_food.append(food_nonaromas_cake['salt'][0])
        piquancy_list_food.append(food_nonaromas_cake['piquant'][0])
        fattiness_list_food.append(food_nonaromas_cake['fat'][0])
        bitterness_list_food.append(food_nonaromas_cake['bitter'][0])
    if food_name == 'cheddar and gruyere':
        sweetness_list_food.append(food_nonaromas_cheddar['sweet'][0])
        acidity_list_food.append(food_nonaromas_cheddar['acid'][0])
        salinity_list_food.append(food_nonaromas_cheddar['salt'][0])
        piquancy_list_food.append(food_nonaromas_cheddar['piquant'][0])
        fattiness_list_food.append(food_nonaromas_cheddar['fat'][0])
        bitterness_list_food.append(food_nonaromas_cheddar['bitter'][0])
    if food_name == 'chicken and turkey':
        sweetness_list_food.append(food_nonaromas_chicken['sweet'][0])
        acidity_list_food.append(food_nonaromas_chicken['acid'][0])
        salinity_list_food.append(food_nonaromas_chicken['salt'][0])
        piquancy_list_food.append(food_nonaromas_chicken['piquant'][0])
        fattiness_list_food.append(food_nonaromas_chicken['fat'][0])
        bitterness_list_food.append(food_nonaromas_chicken['bitter'][0])
    if food_name == 'chilis and hot spicy foods':
        sweetness_list_food.append(food_nonaromas_spicy['sweet'][0])
        acidity_list_food.append(food_nonaromas_spicy['acid'][0])
        salinity_list_food.append(food_nonaromas_spicy['salt'][0])
        piquancy_list_food.append(food_nonaromas_spicy['piquant'][0])
        fattiness_list_food.append(food_nonaromas_spicy['fat'][0])
        bitterness_list_food.append(food_nonaromas_spicy['bitter'][0])
    if food_name == 'chocolate and caramel':
        sweetness_list_food.append(food_nonaromas_chocolate['sweet'][0])
        acidity_list_food.append(food_nonaromas_chocolate['acid'][0])
        salinity_list_food.append(food_nonaromas_chocolate['salt'][0])
        piquancy_list_food.append(food_nonaromas_chocolate['piquant'][0])
        fattiness_list_food.append(food_nonaromas_chocolate['fat'][0])
        bitterness_list_food.append(food_nonaromas_chocolate['bitter'][0])
    if food_name == 'duck, goose and game birds':
        sweetness_list_food.append(food_nonaromas_duck['sweet'][0])
        acidity_list_food.append(food_nonaromas_duck['acid'][0])
        salinity_list_food.append(food_nonaromas_duck['salt'][0])
        piquancy_list_food.append(food_nonaromas_duck['piquant'][0])
        fattiness_list_food.append(food_nonaromas_duck['fat'][0])
        bitterness_list_food.append(food_nonaromas_duck['bitter'][0])
    if food_name == 'fruit-based desserts':
        sweetness_list_food.append(food_nonaromas_fruit['sweet'][0])
        acidity_list_food.append(food_nonaromas_fruit['acid'][0])
        salinity_list_food.append(food_nonaromas_fruit['salt'][0])
        piquancy_list_food.append(food_nonaromas_fruit['piquant'][0])
        fattiness_list_food.append(food_nonaromas_fruit['fat'][0])
        bitterness_list_food.append(food_nonaromas_fruit['bitter'][0])
    if food_name == "goats' cheese and feta":
        sweetness_list_food.append(food_nonaromas_feta['sweet'][0])
        acidity_list_food.append(food_nonaromas_feta['acid'][0])
        salinity_list_food.append(food_nonaromas_feta['salt'][0])
        piquancy_list_food.append(food_nonaromas_feta['piquant'][0])
        fattiness_list_food.append(food_nonaromas_feta['fat'][0])
        bitterness_list_food.append(food_nonaromas_feta['bitter'][0])
    if food_name == 'lamb':
        sweetness_list_food.append(food_nonaromas_lamb['sweet'][0])
        acidity_list_food.append(food_nonaromas_lamb['acid'][0])
        salinity_list_food.append(food_nonaromas_lamb['salt'][0])
        piquancy_list_food.append(food_nonaromas_lamb['piquant'][0])
        fattiness_list_food.append(food_nonaromas_lamb['fat'][0])
        bitterness_list_food.append(food_nonaromas_lamb['bitter'][0])
    if food_name == 'manchego and parmesan':
        sweetness_list_food.append(food_nonaromas_manchego['sweet'][0])
        acidity_list_food.append(food_nonaromas_manchego['acid'][0])
        salinity_list_food.append(food_nonaromas_manchego['salt'][0])
        piquancy_list_food.append(food_nonaromas_manchego['piquant'][0])
        fattiness_list_food.append(food_nonaromas_manchego['fat'][0])
        bitterness_list_food.append(food_nonaromas_manchego['bitter'][0])
    if food_name == 'meaty and oily fish':
        sweetness_list_food.append(food_nonaromas_meaty_fish['sweet'][0])
        acidity_list_food.append(food_nonaromas_meaty_fish['acid'][0])
        salinity_list_food.append(food_nonaromas_meaty_fish['salt'][0])
        piquancy_list_food.append(food_nonaromas_meaty_fish['piquant'][0])
        fattiness_list_food.append(food_nonaromas_meaty_fish['fat'][0])
        bitterness_list_food.append(food_nonaromas_meaty_fish['bitter'][0])
    if food_name == 'mushrooms':
        sweetness_list_food.append(food_nonaromas_mushroom['sweet'][0])
        acidity_list_food.append(food_nonaromas_mushroom['acid'][0])
        salinity_list_food.append(food_nonaromas_mushroom['salt'][0])
        piquancy_list_food.append(food_nonaromas_mushroom['piquant'][0])
        fattiness_list_food.append(food_nonaromas_mushroom['fat'][0])
        bitterness_list_food.append(food_nonaromas_mushroom['bitter'][0])
    if food_name == 'pork, ham and cold meats':
        sweetness_list_food.append(food_nonaromas_pork['sweet'][0])
        acidity_list_food.append(food_nonaromas_pork['acid'][0])
        salinity_list_food.append(food_nonaromas_pork['salt'][0])
        piquancy_list_food.append(food_nonaromas_pork['piquant'][0])
        fattiness_list_food.append(food_nonaromas_pork['fat'][0])
        bitterness_list_food.append(food_nonaromas_pork['bitter'][0])
    if food_name == 'root vegetables and squashes':
        sweetness_list_food.append(food_nonaromas_root['sweet'][0])
        acidity_list_food.append(food_nonaromas_root['acid'][0])
        salinity_list_food.append(food_nonaromas_root['salt'][0])
        piquancy_list_food.append(food_nonaromas_root['piquant'][0])
        fattiness_list_food.append(food_nonaromas_root['fat'][0])
        bitterness_list_food.append(food_nonaromas_root['bitter'][0])
    if food_name == 'salads and green vegetables':
        sweetness_list_food.append(food_nonaromas_salad['sweet'][0])
        acidity_list_food.append(food_nonaromas_salad['acid'][0])
        salinity_list_food.append(food_nonaromas_salad['salt'][0])
        piquancy_list_food.append(food_nonaromas_salad['piquant'][0])
        fattiness_list_food.append(food_nonaromas_salad['fat'][0])
        bitterness_list_food.append(food_nonaromas_salad['bitter'][0])
    if food_name == 'shellfish, crab and lobster':
        sweetness_list_food.append(food_nonaromas_shellfish['sweet'][0])
        acidity_list_food.append(food_nonaromas_shellfish['acid'][0])
        salinity_list_food.append(food_nonaromas_shellfish['salt'][0])
        piquancy_list_food.append(food_nonaromas_shellfish['piquant'][0])
        fattiness_list_food.append(food_nonaromas_shellfish['fat'][0])
        bitterness_list_food.append(food_nonaromas_shellfish['bitter'][0])
    if food_name == 'tomato-based dishes':
        sweetness_list_food.append(food_nonaromas_tomato['sweet'][0])
        acidity_list_food.append(food_nonaromas_tomato['acid'][0])
        salinity_list_food.append(food_nonaromas_tomato['salt'][0])
        piquancy_list_food.append(food_nonaromas_tomato['piquant'][0])
        fattiness_list_food.append(food_nonaromas_tomato['fat'][0])
        bitterness_list_food.append(food_nonaromas_tomato['bitter'][0])
    if food_name == 'white fish':
        sweetness_list_food.append(food_nonaromas_white_fish['sweet'][0])
        acidity_list_food.append(food_nonaromas_white_fish['acid'][0])
        salinity_list_food.append(food_nonaromas_white_fish['salt'][0])
        piquancy_list_food.append(food_nonaromas_white_fish['piquant'][0])
        fattiness_list_food.append(food_nonaromas_white_fish['fat'][0])
        bitterness_list_food.append(food_nonaromas_white_fish['bitter'][0])

9747it [00:00, 19731.66it/s]


In [113]:
pair_df_test = pair_df.copy()
pair_df_test['sweetness_food'] = sweetness_list_food
pair_df_test['acidity_food'] = acidity_list_food
pair_df_test['salinity_food'] = salinity_list_food
pair_df_test['piquancy_food'] = piquancy_list_food
pair_df_test['fattiness_food'] = fattiness_list_food
pair_df_test['bitterness_food'] = bitterness_list_food

In [115]:
pair_df_test.head(3)

,Pairing Food,Grape,Popularity,Critics' Score,Avg. Price kr / 750ml,Wine,Region,Country,name_pair,sweetness,acidity,piquancy,fattiness,bitterness,Alcohol,Price_Review,Vintage,name_wine_review,sweetness_food,acidity_food,salinity_food,piquancy_food,fattiness_food,bitterness_food
0,beef and venison,Bordeaux Blend Red,1st,96 / 100,"7,926",Chateau Mouton Rothschild,Pauillac,France,Bordeaux Blend Red Chateau Mouton Rothschild P...,-27.316847,-0.000972,-5.65303,-4.383264,-10.966054,14%,$675,2016.0,Bordeaux-style Red Blend ChÃ¢teau Mouton Roths...,0.141897,0.318145,0.655264,0.473096,0.420863,0.228206
1,beef and venison,Bordeaux Blend Red,3rd,96 / 100,"10,564",Chateau Lafite Rothschild,Pauillac,France,Bordeaux Blend Red Chateau Lafite Rothschild P...,-0.000151,-0.000972,-5.65303,-4.383264,-10.966054,NaN,$869,2016.0,Bordeaux-style Red Blend ChÃ¢teau Lafite Roths...,0.141897,0.318145,0.655264,0.473096,0.420863,0.228206
2,beef and venison,Merlot,4th,96 / 100,"47,971",Petrus,Pomerol,France,Merlot Petrus Pomerol France,-0.000151,-3.860945,-5.65303,-4.383264,27.357079,14%,$65,2014.0,Merlot ChÃ¢teau Fayat Pomerol France,0.141897,0.318145,0.655264,0.473096,0.420863,0.228206


In [116]:
pair_df_test.to_csv('pairing_food_final.csv', index=False)